In [138]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
import urllib.request
import time
from math import ceil

In [222]:
"""
@hi: resmin yüksekliği
@ph: cismin resimdeki bulunduğu piksel
@Z: Kameranın yere uzaklığı
"""
def calcRealY(hi, ph, Z):
    # Kamera teknik sabitleri
    hs = 0.00274 # m (2.74mm) 
    f = 0.00329 #m (3.29mm)

    # Cismin resimdeki orta noktaya göre konumu
    pv = hi/2 - ph # px

    # Cismin sensör alanındaki orta noktaya göre konumu
    ys = pv * hs / hi # m

    # Cismin gerçek konumu
    Y = ys * Z / f
    return Y

"""
@hi: resmin yüksekliği
@ph: cismin resimdeki bulunduğu piksel
@Z: Kameranın yere uzaklığı
"""
def calcRealX(hi, ph, Z):
    # Kamera teknik sabitleri
    hs = 0.00367 # m (2.74mm) 
    f = 0.00360 #m (3.29mm)

    # Cismin resimdeki orta noktaya göre konumu
    pv = ph - hi/2  # px

    # Cismin sensör alanındaki orta noktaya göre konumu
    ys = pv * hs / hi # m

    # Cismin gerçek konumu
    Y = ys * Z / f
    return Y


In [210]:
def draw_grid(img, line_color=(0, 255, 0), thickness=1, type_=cv2.LINE_AA, pxstep=50):
    '''(ndarray, 3-tuple, int, int) -> void
    draw gridlines on img
    line_color:
        BGR representation of colour
    thickness:
        line thickness
    type:
        8, 4 or cv2.LINE_AA
    pxstep:
        grid line frequency in pixels
    '''
    x = pxstep
    y = pxstep
    while x < img.shape[1]:
        cv2.line(img, (x, 0), (x, img.shape[0]), color=line_color, lineType=type_, thickness=thickness)
        x += pxstep

    while y < img.shape[0]:
        cv2.line(img, (0, y), (img.shape[1], y), color=line_color, lineType=type_, thickness=thickness)
        y += pxstep

In [141]:
def drawPictureCoor(img):
    w = img.shape[1]
    h = img.shape[0]
    color1 = (255,0,0)
    color2 = (0,0,255)
    # Merkez koordinat
    cv2.line(img, (ceil(w/2), 0), (ceil(w/2), h), color1)
    cv2.line(img, (0, ceil(h/2)), (w, ceil(h/2)), color1)
    
    # Ana koordinat
    cv2.line(img, (0, 0), (w, 0), color2)
    cv2.line(img, (0, 0), (0, h), color2)
    

In [142]:
def drawLineToObj(img, pos):
    w = img.shape[1]
    h = img.shape[0]
    color1 = (255,0,0)
    color2 = (0,0,255)
    fsize = .7
    thick = 1
    
    # Merkeze göre konumuna ok çizip konumu yazdırılır
    cv2.line(img, (ceil(w/2), ceil(h/2)), (pos[0], pos[1]), color1)
    cv2.putText(img, "(0,0)", (ceil(w/2),ceil(h/2)), cv2.FONT_HERSHEY_SIMPLEX, fsize, 
                 color1, thick, cv2.LINE_AA, False)
    cv2.putText(img, "("+str(pos[0]-ceil(w/2))+","+str(ceil(h/2)-pos[1])+")", (pos[0],pos[1]), cv2.FONT_HERSHEY_SIMPLEX, fsize, 
                 color1, thick, cv2.LINE_AA, False)
    
    # Resmin hangi pikselde olduğu yazdırılır
    cv2.putText(img, "("+str(pos[0])+","+str(pos[1])+")", (pos[0],pos[1]+25), cv2.FONT_HERSHEY_SIMPLEX, fsize, 
                 color2, thick, cv2.LINE_AA, False)

In [143]:
def drawRealPos(img, pos, size):
    w = img.shape[1]
    h = img.shape[0]
    color = (0, 255,0)
    fsize = .7
    thick = 1
    
    # X ekseni çizilir
    cv2.line(img, (ceil(w/2), ceil(h/2)), (pos[0], ceil(h/2)), color)
    cv2.putText(img, ("%.2f" % (size[0]*100)) + "cm", (pos[0], ceil(h/2)), cv2.FONT_HERSHEY_SIMPLEX, fsize, 
                 color, thick, cv2.LINE_AA, False)
    
    # Y ekseni çizilir
    cv2.line(img, (ceil(w/2), ceil(h/2)), (ceil(w/2), pos[1]), color)
    cv2.putText(img, ("%.2f" % (size[1]*100)) + "cm", (ceil(w/2),pos[1]), cv2.FONT_HERSHEY_SIMPLEX, fsize, 
                 color, thick, cv2.LINE_AA, False)

In [250]:
def streamAndProcess(source, grid=False, size=1):
    #stream = urllib.request.urlopen('http://192.168.43.101:8000/stream.mjpg')
    stream = urllib.request.urlopen(source)
    myImage = ""
    firstIter = True
    bytes = b''
    obj = [232,180]
    display_handle=display(None, display_id=True)

    try:
        while True:
            start = time.time()
            bytes += stream.read(1024)
            a = bytes.find(b'\xff\xd8') #frame starting 
            b = bytes.find(b'\xff\xd9') #frame ending
            if a != -1 and b != -1:
                jpg = bytes[a:b+2]
                bytes = bytes[b+2:]
                img = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
                drawPictureCoor(img)
                drawLineToObj(img,obj)
                size0 = calcReal2(img.shape[1], obj[0], size)
                size1 = calcReal(img.shape[0], obj[1], size)
                drawRealPos(img, obj, [size0,size1])
                if(grid):
                    draw_grid(img)
                encIm = cv2.imencode(".jpeg", img)[1]
                asd = Image(data=encIm.tobytes())
                display_handle.update(asd)
            
            print("Geçen süre: %.4f" % (time.time() - start))
    except KeyboardInterrupt:
        pass
    finally:
        display_handle.update(None)
                                

In [ ]:
streamAndProcess('http://192.168.137.61:8000/stream.mjpg', size=8.60)
# 8.60
#streamAndProcess('http://192.168.137.250:8080/video')